In [1]:
!pip install wandb

  Using cached wandb-0.12.5-py2.py3-none-any.whl (1.7 MB)
     |████████████████████████████████| 1.7 MB 1.3 MB/s eta 0:00:01
You should consider upgrading via the '/home/user/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:
!pip install torchvision==0.6.0

In [156]:
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
from tqdm import tqdm

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

from torch_geometric.nn import Node2Vec
from torch_geometric.data import Data

import networkx as nx

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.manifold import TSNE

from scipy.spatial import distance

import matplotlib.pyplot as plt

import wandb 

In [46]:
graph_init = pd.read_csv("graph_abakan_init.csv", index_col = 0)

,ts,version,year_month,edge_id,fiji_id,seg_begin,seg_end,tile_from,tile_to,node_from,node_to,length,width,azimuth_begin,azimuth_end,road_class,sm_class,rm_car_class,rm_car_speed,type,light_begin,light_end,zlevel_from,zlevel_to,turn_to,payments_flags,is_city_edge,barrier,turn_restrictions,quartal_id,truck_zone_id,traverse,passability,pedo_offset,bad_road,alt_road,style,lanes,def_speed,road_exit,street,geom,sign_segment_begin,sign_segment_end
city,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
abakan,1607674440,2020-12_1607674440,202012,19423207912112130,9711603956056065,0,1,296624533920,296624533920,1.091739e+17,1.091751e+17,104,7,42,42,6,18,8,90,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,7,2,90,0,NaN,"LINESTRING(91.337229694 53.5716834537,91.33723...",0,1
abakan,1607674440,2020-12_1607674440,202012,19423207912112131,9711603956056065,0,1,296624533920,296624533920,1.091751e+17,1.091739e+17,104,7,222,222,6,18,8,90,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,7,2,90,0,NaN,"LINESTRING(91.3382073305 53.5724154795,91.3372...",0,1
abakan,1607674440,2020-12_1607674440,202012,19423207912112132,9711603956056066,0,1,296624533934,296624553972,1.094721e+17,1.094652e+17,553,3,324,318,2,8,8,90,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,7,1,90,0,NaN,"LINESTRING(91.5867024485 53.5748962611,91.5866...",0,1
abakan,1607674440,2020-12_1607674440,202012,19423207912112133,9711603956056066,0,1,296624553972,296624533934,1.094652e+17,1.094721e+17,553,3,138,144,2,8,8,90,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,7,1,90,0,NaN,"LINESTRING(91.5809012182 53.5784671832,91.5841...",0,1
abakan,1607674440,2020-12_1607674440,202012,19423207912112134,9711603956056067,0,1,296624533911,296624533910,1.089704e+17,1.089663e+17,500,7,210,210,6,18,8,90,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,7,2,90,0,NaN,"LINESTRING(91.1669185484 53.5731552273,91.1669...",0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
abakan,1607674440,2020-12_1607674440,202012,140060152502123605,70030076251061802,0,1,296624774400,296624774400,1.096813e+17,1.096815e+17,91,3,6,6,2,2,5,60,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,7,1,60,0,NaN,"LINESTRING(91.7617266052 53.6960142902,91.7619...",0,1
abakan,1607674440,2020-12_1607674440,202012,140060152502123620,70030076251061810,0,1,296624774400,296624774400,1.096813e+17,1.096839e+17,144,3,96,96,2,2,5,60,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,7,1,60,0,"ru_nom='Тобольская',ru_dat='Тобольская',ru_acc...","LINESTRING(91.7617168135 53.6960097694,91.7638...",0,1
abakan,1607674440,2020-12_1607674440,202012,140060152502123621,70030076251061810,0,1,296624774400,296624774400,1.096839e+17,1.096813e+17,144,3,276,276,2,2,5,60,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,7,1,60,0,"ru_nom='Тобольская',ru_dat='Тобольская',ru_acc...","LINESTRING(91.763899001 53.6958676549,91.76171...",0,1


In [293]:
graph = pd.read_csv("graph_abakan_F_encoded.csv", index_col = 0)
X = pd.read_csv("abakan_full_routes_final_weather_L_NaN_filtered_FIXED.csv", index_col = 0).dropna().drop(["real_dist", "speed", "time", "length", "route_type"], axis = 1).reset_index(drop=True)

Clearing data from the noised objects:

In [294]:
X = X[(X["rebuildCount"] <= 1) & (X["RTA"] > 30) & (X["RTA"] < 3000)]
time = X["start_timestamp"].reset_index(drop = True)
X = X.drop(["rebuildCount", "start_timestamp"], axis = 1)
X.reset_index(drop = True, inplace = True)

In [ ]:
route_edges = np.array(X["edges"].map(lambda x: x.replace("'", "")))

In [ ]:
wind_dir_classes = pd.get_dummies(X.wind_dir, prefix='wind_dir_class')
day_classes = pd.get_dummies(X.day_period, prefix='day_class')
Y = X["RTA"]
X = X.join(wind_dir_classes).join(day_classes).drop(["edges", "wind_dir", "day_period", "RTA"], axis = 1)

In [ ]:
graph_features = graph.drop(["edge_id", "adjacent"], axis = 1)
embs = np.array(graph_features)

### Spatial graph and auxiliary structures construction

In [188]:
splited = [graph_init["geom"][i].split("(") for i in range(len(graph_init["geom"]))]
coords = [splited[i][1] for i in range(len(splited))]
first_coord = [coords[i].split(",")[0] for i in range(len(coords))] 
first_coord = [list(map(float, first_coord[i].split(" "))) for i in range(len(first_coord))]
first_coord = np.array(first_coord)

In [189]:
max(first_coord[:, 0]), min(first_coord[:, 0]), max(first_coord[:, 1]), min(first_coord[:, 1])

(91.9791699282, 90.5756827322)

In [198]:
grid_size = 30
G = nx.grid_2d_graph(grid_size, grid_size)

In [199]:
fc = np.linspace(min(first_coord[:, 0]), max(first_coord[:, 0]), num = grid_size)
sc = np.linspace(min(first_coord[:, 1]), max(first_coord[:, 1]), num = grid_size)

grid = np.array([[[fc[i], sc[j]] for i in range(len(fc))] for j in range(len(sc))])

In [200]:
for i in range(grid_size):
    for j in range(grid_size):
        G.nodes[i, j]["coord"] = grid[i][j]

In [201]:
locations = [int(route_edges[i].split(",")[0]) for i in range(len(route_edges))]

In [206]:
encoded_positions = []
for i in tqdm(range(len(locations))):
    location = first_coord[graph[graph["edge_id"] == locations[i]].index[0]]
    min_dist, encoded_position = 10, 0
    for j in range(grid_size):
        for k in range(grid_size):
            dist = distance.euclidean(location, G.nodes[j, k]["coord"])
            if dist < min_dist:
                min_dist = dist
                encoded_position = [j,k]
    encoded_positions.append(encoded_position)

100%|██████████| 82202/82202 [11:02<00:00, 124.09it/s]


In [298]:
encoded_positions = np.array(encoded_positions)
encoded_positions.shape

(82202, 2)

### Temporal graph construction and auxiliary structures construction

In [282]:
T = nx.grid_2d_graph(4, 7)
for i in range(3):
    T.add_node((4, i))
    
T.nodes[4, 0]["time"] = 29
T.nodes[4, 1]["time"] = 30
T.nodes[4, 2]["time"] = 31

In [283]:
T.add_edge((4, 0), (4, 1))
T.add_edge((4, 0), (3, 0))

T.add_edge((4, 1), (4, 2))
T.add_edge((4, 1), (3, 1))

T.add_edge((4, 2), (3, 2))

In [284]:
date = 1
for i in range(4):
    for j in range(7):
        G.nodes[i, j]["time"] = date
        date +=1

In [291]:
time

2         2020-12-06 12:21:17.0
3         2020-12-13 14:25:20.0
4         2020-12-04 17:43:41.0
5         2020-12-15 17:40:56.0
6         2020-12-02 09:42:09.0
                  ...          
122304    2020-12-01 15:37:06.0
122305    2020-12-13 03:03:59.0
122306    2020-12-29 10:02:33.0
122308    2020-12-18 16:52:58.0
122310    2020-12-24 00:22:25.0
Name: start_timestamp, Length: 82202, dtype: object

In [295]:
times = [int(time[i][8:10]) for i in range(len(time))]

In [296]:
encoded_dates = []
for i in range(len(times)):
    encoded_dates.append([int((times[i] - 1) / 7), (times[i] - 1) % 7])

In [297]:
encoded_dates = np.array(encoded_dates)
encoded_dates.shape

(82202, 2)

### Node2Vec for road network link embeddings

In [373]:
train_mask = []
test_mask = []
for i in range(len(graph)):
    test_mask.append(False)
    train_mask.append(False)
for i in range(0, int(len(graph)*4/5)):
    train_mask[i] = True
for i in range(int(len(graph)*4/5), len(graph)):
    test_mask[i] = True

In [374]:
def get_index_tuples(df):
    lst = []
    for i in tqdm(range(len(df))):
        tmp = df['adjacent'][i].split(',')
        for edge in tmp:
            lst.append([i, int(edge)])
            
    return lst

lst = pd.DataFrame(get_index_tuples(graph))
edge_index = torch.tensor(np.array(lst).T, dtype=torch.long)

x = np.array(graph.drop(["edge_id", "adjacent"], axis = 1))
x = torch.tensor(x, dtype=torch.float)

data = Data(x=x, edge_index=edge_index, train_mask = train_mask, test_mask = test_mask)

100%|██████████| 65524/65524 [00:00<00:00, 81444.92it/s] 


In [375]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Node2Vec(data.edge_index, embedding_dim=40, walk_length=3,
                 context_size=2, walks_per_node=2,
                 num_negative_samples=1, p=1, q=1, sparse=True).to(device)

loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

for epoch in range(1, 21):
    loss = train()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

Epoch: 01, Loss: 5.0219
Epoch: 02, Loss: 3.8428
Epoch: 03, Loss: 2.9831
Epoch: 04, Loss: 2.5703
Epoch: 05, Loss: 2.3563
Epoch: 06, Loss: 2.2037
Epoch: 07, Loss: 2.0942
Epoch: 08, Loss: 2.0004
Epoch: 09, Loss: 1.9276
Epoch: 10, Loss: 1.8636
Epoch: 11, Loss: 1.8095
Epoch: 12, Loss: 1.7561
Epoch: 13, Loss: 1.7035
Epoch: 14, Loss: 1.6555
Epoch: 15, Loss: 1.6070
Epoch: 16, Loss: 1.5631
Epoch: 17, Loss: 1.5302
Epoch: 18, Loss: 1.4902
Epoch: 19, Loss: 1.4573
Epoch: 20, Loss: 1.4298


In [382]:
model.eval()
link_embeddings = model(torch.arange(data.num_nodes, device=device))

In [383]:
link_embeddings = link_embeddings.cpu().detach().numpy()

In [371]:
start_links_ids = [graph[graph["edge_id"] == locations[i]].index[0] for i in range(len(locations))]

In [388]:
link_embeddings_wrt_routes = link_embeddings[start_links_ids]

In [28]:
# model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)

Using cache found in /home/jovyan/.cache/torch/hub/pytorch_vision_v0.6.0


### MURAT model

In [479]:
import math 
class LaplacianEmbedding(nn.Module):
    def __init__(self, vertexes_number, embedding_size = 20):
        super(LaplacianEmbedding, self).__init__()
        self.vertexes_number, self.embedding_size = vertexes_number, embedding_size
        self.weights = torch.Tensor(self.vertexes_number, self.embedding_size)
        self.weights = nn.Parameter(self.weights)
        nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5))

    def forward(self, A):
        result = torch.Tensor([0]).cuda()
        for i in tqdm(range(A.shape[0])):
            for j in range(A.shape[1]):
                result += A[i][j]*torch.norm(self.weights[i] - self.weights[j])
        return result, self.weights

Laplacian embedding test:

In [480]:
laplacian_test = LaplacianEmbedding(31)
temporal_adj_matrix = np.array(nx.adjacency_matrix(T).todense())
laplacian_test(temporal_adj_matrix)[1].shape

100%|██████████| 31/31 [00:00<00:00, 629.35it/s]


torch.Size([31, 20])

In [481]:
laplacian_test(temporal_adj_matrix)[0]

100%|██████████| 31/31 [00:00<00:00, 643.07it/s]


tensor([82.7973], device='cuda:0', grad_fn=<AddBackward0>)

In [361]:
# There are no information available about what residual architecture MURAT authors implemented

class ResLinearBlock(nn.Module):
    def __init__(self):
        super(ResLinearBlock, self).__init__()
        self.linear_1 = nn.Linear(1024, 1024)
        self.linear_2 = nn.Linear(1024, 1024)
        self.batch_norm_1 = nn.BatchNorm1d(1024)
        self.batch_norm_2 = nn.BatchNorm1d(1024)

    def forward(self, x):
        result = self.linear_1(x)
        result = F.relu(self.batch_norm_1(result))
        result = self.linear_2(result)
        result = F.relu(self.batch_norm_2(result))
        
        return x + result

Residual embedding test:

In [363]:
res_test = ResLinearBlock()
test_input = torch.ones(2, 1024)
res_test(test_input)

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], grad_fn=<AddBackward0>)

In [482]:
class MURAT(nn.Module):
    def __init__(self):
        super(MURAT, self).__init__()
        
        # Spatial and temporal graph embeddings 
        self.spatial_emb = LaplacianEmbedding(30*30)
        self.temporal_emb = LaplacianEmbedding(31)
        
        # Convert to residual network input
        self.linear_0 = nn.Linear(20 + 20 + 40, 1024)
        
        # Residual network
        self.res_block_1 = ResLinearBlock()
        self.res_block_2 = ResLinearBlock()
        self.res_block_3 = ResLinearBlock()
        self.res_block_4 = ResLinearBlock()
        
        # Final part
        self.linear_1 = nn.Linear(1024, 1)      
        
    def forward(self, link_embeddings, spatial_adj_matrix, spatial_indices, temporal_adj_matrix, temporal_indices):
        summ_s, spatial = self.spatial_emb(spatial_adj_matrix)
        summ_t, temporal = self.temporal_emb(temporal_adj_matrix)
        print(summ_s.device, summ_t.device)
        spatial_matrix = [spatial[spatial_indices[i][0]*30 + spatial_indices[i][1]] for i in range(len(spatial_indices))]
        temporal_matrix = [temporal[temporal_indices[i][0]*7 + temporal_indices[i][1]] for i in range(len(temporal_indices))]
        
        spatial_tensor = torch.stack(spatial_matrix, dim = 0)
        temporal_tensor = torch.stack(temporal_matrix, dim = 0)
        
        resnet_input = torch.cat([link_embeddings, spatial_tensor, temporal_tensor], dim = 1)
        resnet_input = self.linear_0(resnet_input)
        
        resnet_output = self.res_block_1(resnet_input)
        resnet_output = self.res_block_2(resnet_output)
        resnet_output = self.res_block_3(resnet_output)
        resnet_output = self.res_block_4(resnet_output)
        
        final_output = F.relu(self.linear_1(resnet_output))
        print(summ_s.device, summ_t.device)
        return final_output, summ_s, summ_t

In [483]:
num_epochs = 200
learning_rate = 0.01

device = ""
gpu_ids = []
if torch.cuda.is_available():
    gpu_ids += [gpu_id for gpu_id in range(torch.cuda.device_count())]
    device = torch.device(f'cuda:{gpu_ids[0]}')
    torch.cuda.set_device(device)
else:
    device = torch.device('cpu')
print(device)
#device = "cuda"

model = MURAT()
model = model.to(device)

criterion = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

cuda:0


In [484]:
ratio = 0.8
train_mask = {i for i in range(int(ratio*82200), 120000)} 
test_mask = {i for i in range(int(ratio*82200))}

In [485]:
max(encoded_dates[:, 1])

6

In [486]:
trainY = torch.Tensor(Y).cuda()

In [487]:
link_embs_tensor = torch.Tensor(link_embeddings_wrt_routes).cuda()
link_embs_tensor.shape

torch.Size([82202, 40])

In [488]:
s.device

device(type='cpu')

In [ ]:
temporal_adj_matrix = np.array(nx.adjacency_matrix(T).todense())
spatial_adj_matrix = np.array(nx.adjacency_matrix(G).todense())

alpha = 0.2

model.train()
for epoch in range(num_epochs):
    outputs, s, t = model(link_embs_tensor, spatial_adj_matrix, encoded_positions, temporal_adj_matrix, encoded_dates)
    optimizer.zero_grad()
    loss = criterion(outputs, trainY.unsqueeze(-1)) + alpha*s + alpha*t
    loss.backward()
    optimizer.step()
    print(epoch, loss)

100%|██████████| 31/31 [00:00<00:00, 452.80it/s]


cuda:0 cuda:0
cuda:0 cuda:0
0 tensor([918.5672], device='cuda:0', grad_fn=<AddBackward0>)


100%|██████████| 31/31 [00:00<00:00, 344.43it/s]


cuda:0 cuda:0
cuda:0 cuda:0
1 tensor([857.8806], device='cuda:0', grad_fn=<AddBackward0>)


100%|██████████| 31/31 [00:00<00:00, 426.72it/s]


...

100%|██████████| 31/31 [00:00<00:00, 472.15it/s]


cuda:0 cuda:0
cuda:0 cuda:0
12 tensor([365.1529], device='cuda:0', grad_fn=<AddBackward0>)


100%|██████████| 31/31 [00:00<00:00, 459.99it/s]


cuda:0 cuda:0
cuda:0 cuda:0
13 tensor([344.8575], device='cuda:0', grad_fn=<AddBackward0>)


  4%|▎         | 33/900 [00:02<01:04, 13.37it/s]

In [449]:
%debug

> /home/user/conda/lib/python3.7/site-packages/torch/nn/functional.py(1610)linear()
   1608     if input.dim() == 2 and bias is not None:
   1609         # fused op is marginally faster
-> 1610         ret = torch.addmm(bias, input, weight.t())
   1611     else:
   1612         output = input.matmul(weight.t())

ipdb> exit


In [ ]:
link_embeddings, spatial_adj_matrix, spatial_indices, temporal_adj_matrix, temporal_indices

In [404]:
epoch

0

In [113]:
wdr7.train()
for epoch in range(num_epochs):
    losses = []
    overall_loss, loss_test = 0, 0
    
    for i in tqdm(range(len(batch_list))):
        indices = np.array([batch_list[i][j][2] for j in range(len(batch_list[i]))])
        indices = np.array(list(set(indices) - train_mask))
        if len(indices) == 0:
            continue
        trainX = Variable(torch.Tensor(np.array([batch_list[i][j][0] for j in range(len(batch_list[i])) if batch_list[i][j][2] in indices])), requires_grad=True).to(device)
        trainY = Variable(torch.Tensor(np.array([batch_list[i][j][1] for j in range(len(batch_list[i])) if batch_list[i][j][2] in indices])), requires_grad=True).to(device)
        outputs = wdr7(x = trainX, deep_input = deep_input, wide_input = wide_input, indices = indices)
        optimizer.zero_grad()
#        print(torch.autograd.grad(outputs=outputs[0], inputs=wo, retain_graph=True)[0][0])
        # obtain the loss function
        loss = criterion(outputs, trainY.unsqueeze(-1))
        loss.backward(retain_graph=True)
        optimizer.step()
        losses.append(loss.item()/trainX.shape[0])
        overall_loss += loss.item()
    
    for i in tqdm(range(len(batch_list))):
        indices = np.array([batch_list[i][j][2] for j in range(len(batch_list[i]))])
        indices = np.array(list(set(indices) - test_mask))
        if len(indices) == 0:
            continue
        testX = Variable(torch.Tensor(np.array([batch_list[i][j][0] for j in range(len(batch_list[i])) if batch_list[i][j][2] in indices])), requires_grad=True).to(device)
        testY = Variable(torch.Tensor(np.array([batch_list[i][j][1] for j in range(len(batch_list[i])) if batch_list[i][j][2] in indices])), requires_grad=True).to(device)
        outputs = wdr7(x = testX, deep_input = deep_input, wide_input = wide_input, indices = indices)
        loss = criterion(outputs, testY.unsqueeze(-1))
        loss_test += loss.item()
        
    print(epoch, {"train_MAE": overall_loss/len(batch_list)}, {"test_MAE": loss_test/len(batch_list)})
    wandb.log({"train_MAE":  overall_loss/len(batch_list),
              "test_MAE": loss_test/len(batch_list)})

100%|██████████| 225/225 [00:13<00:00, 17.02it/s]


0 {'train_MAE': 489.5816154649523} {'test_MAE': 273.5765378994412}


100%|██████████| 225/225 [00:13<00:00, 16.42it/s] 


1 {'train_MAE': 328.7494549899631} {'test_MAE': 369.3745949130588}


100%|██████████| 225/225 [00:14<00:00, 15.19it/s] 


...

100%|██████████| 225/225 [00:13<00:00, 16.17it/s] 


197 {'train_MAE': 306.85791870117185} {'test_MAE': 199.36454530504014}


100%|██████████| 225/225 [00:13<00:00, 16.40it/s] 


198 {'train_MAE': 261.31643498738606} {'test_MAE': 197.58060406155056}


 76%|███████▌  | 170/225 [00:23<00:05, 10.48it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

